<a href="https://colab.research.google.com/github/HJJunn/DeepLearning---NLP/blob/main/14_RNN_%EC%9D%B8%EC%BD%94%EB%8D%94_%EB%94%94%EC%BD%94%EB%8D%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
lines = pd.read_csv("/content/fra_eng.csv")
lines.sample(10)

,src,tar
55813,Are you in a bad mood?,Es-tu de mauvaise humeur ?
2305,I'm unwell.,Je ne me sens pas bien.
57629,I can't show you this.,Je ne peux pas vous le montrer.
16032,I taught French.,J'ai enseigné le français.
35018,I'm a little upset.,Je suis un peu en colère.
46217,What's all the fuss?,C'est quoi tout ce raffut ?
1306,I'm tough.,Je suis dur.
54449,Were you with anyone?,Étais-tu avec qui que ce soit ?
26926,I didn't go there.,Je ne suis pas allé là.
14263,You're elusive.,Tu es insaisissable.


In [ ]:
lines

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
59995,It won't be that hard.,Ce ne sera pas si difficile.
59996,It won't be that hard.,Ce ne sera pas si dur.
59997,It won't happen again.,Ça n'arrivera plus.
59998,It won't happen again.,Ça ne se produira plus.


In [ ]:
# <sos> -> \t <eos> -> \n
lines.tar = lines.tar.apply(lambda x: '\t' + x + '\n')
lines.sample(10)

,src,tar
8676,Isn't it cute?,\tN'est-il pas mignon ?\n
1075,Head west.,\tDirige-toi vers l'ouest.\n
36856,The doors are shut.,\tLes portes sont fermées.\n
8263,I was bullied.,\tJ'ai subi des brimades.\n
31086,We're all at risk.,\tNous sommes tous en danger.\n
30122,This is not funny.,\tCe n'est pas drôle !\n
49469,I hard-boiled an egg.,\tJ'ai cuit un œuf dur.\n
37850,Tom saw Mary leave.,\tTom vit Mary partir.\n
42134,I think he is right.,\tJe pense qu'il a raison.\n
50118,I think Tom can help.,\tJe pense que Tom peut vous aider.\n


In [ ]:
# 문자 집합 구축

src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [ ]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1

print(src_vocab_size)
print(tar_vocab_size)

80
102


In [ ]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
# 문자에 인덱스를 부여

src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 3

In [ ]:
#영어 문장 샘플 인코딩
encoder_input = []
for line in lines.src:
    encoded_line = []
    for char in line:
        encoded_line.append(src_to_index[char])
    encoder_input.append(encoded_line)
print("socure 문장의 정수 인코딩:", encoder_input[:5])

socure 문장의 정수 인코딩: [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [ ]:
#프랑스 문장 샘플 인코딩
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 46, 50, 3, 4, 2], [1, 37, 50, 67, 52, 57, 54, 12, 2], [1, 29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 2], [1, 26, 64, 70, 56, 54, 3, 4, 2], [1, 43, 50, 61, 70, 69, 3, 4, 2]]


In [ ]:
#프랑스 문장 실제값 정수 인코딩
#실제값 : 마지막에 심볼 <eos> 만 들어있음
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[46, 50, 3, 4, 2], [37, 50, 67, 52, 57, 54, 12, 2], [29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 2], [26, 64, 70, 56, 54, 3, 4, 2], [43, 50, 61, 70, 69, 3, 4, 2]]


In [ ]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])

print('source 문장의 최대 길이:', max_src_len)
print('target 문장의 최대 길이:', max_tar_len)

source 문장의 최대 길이: 22
target 문장의 최대 길이: 74


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen = max_src_len, padding = 'post')
decoder_input = pad_sequences(decoder_input, maxlen = max_tar_len, padding = 'post')
decoder_target = pad_sequences(decoder_target, maxlen = max_tar_len, padding = 'post')

In [ ]:
# encoder_input = to_categorical(encoder_input)
# decoder_input = to_categorical(decoder_input)
# decoder_target = to_categorical(decoder_target)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
#인코더
encoder_inputs = Input(shape=(None,))  # (batch, time)
encoder_embedding = Embedding(src_vocab_size, 128)
encoder_emb= encoder_embedding(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units = 256, return_state = True)(encoder_emb)

# 은닉상태와 셀 상태
encoder_states = [state_h, state_c]

In [ ]:
#디코더
decoder_inputs = Input(shape = (None,))
decoder_embedding = Embedding(tar_vocab_size, 128)
decoder_emb = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state = True)
# 디코더에게 인코더의 마지막 은닉상태, 셀 상태를 전달
decoder_outputs, _,_ = decoder_lstm(decoder_emb, initial_state = encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer = "rmsprop", loss = "sparse_categorical_crossentropy")


In [ ]:
import numpy as np

# decoder_target shape: (samples, timesteps)
decoder_target = np.array(decoder_target)

model.fit([encoder_input, decoder_input],
          decoder_target,
          batch_size=32,
          epochs=10,
          validation_split=0.2)


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - loss: 0.8284 - val_loss: 0.6356
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - loss: 0.4876 - val_loss: 0.5321
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 0.4105 - val_loss: 0.4810
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 0.3717 - val_loss: 0.4488
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 0.3440 - val_loss: 0.4275
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 0.3248 - val_loss: 0.4127
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - loss: 0.3122 - val_loss: 0.3999
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 0.2997 - val_loss: 0.3851
Epoch 9/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 0.2865 - val_loss: 0.3773
Epoch 10/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 0.2773 - val_loss: 0.3699


In [ ]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(60000, 22)
(60000, 74)
(60000, 74)


## 3) seq2seq 기계 번역기 동작시키기


*   번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻습니다.
*   상태와 [SOS]에 해당하는 \t를 디코더로 보냅니다.
*   디코더가 [EOS]에 해당하는 \n이 나올 때까지 다음 문자를 예측하는 행동을 반복합니다.



In [ ]:
#인코더
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)

In [ ]:
#디코더
#이전 시점의 상태들을 저장하는 텐서
decoder_inputs_single = Input(shape = (None,))
decoder_state_input_h = Input(shape = (256,))
decoder_state_input_c = Input(shape = (256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_emb_2 = decoder_embedding(decoder_inputs_single)

#문장의 다음 단어를 예측하기 위해 initial_state를 이전 시점의 상태로 사용
decoder_outputs, state_h, state_c = decoder_lstm(decoder_emb_2, initial_state = decoder_states_inputs)

#은닉상태와 셀 상태를 버리지 않음
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs = [decoder_inputs_single] + decoder_states_inputs, outputs = [decoder_outputs] + decoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # 에 해당하는 정수 인덱스 생성
  target_seq = np.array([[tar_to_index['\t']]])

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # 에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.array([[sampled_token_index]])

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index].strip()[1:-1])
  print('번역 문장:', decoded_sentence.strip()[1:-1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge !
번역 문장: aisse Tom s'en al

# Word-Level 번역기 만들기

In [2]:
import os
import re
import shutil
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [57]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)


ZIP file downloaded to fra-eng.zip


In [58]:
num_samples = 33000

In [59]:
def to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
  # 악센트 제거 함수 호출
  sent = to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 추가.
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent


In [60]:
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print("전처리 전 영어문장:", en_sent)
print("전처리 후 영어문장:", preprocess_sentence(en_sent))
print("전처리 전 프랑스문장:", fr_sent)
print("전처리 후 프랑스 문장:", preprocess_sentence(fr_sent))

전처리 전 영어문장: Have you had dinner?
전처리 후 영어문장: have you had dinner ?
전처리 전 프랑스문장: Avez-vous déjà diné?
전처리 후 프랑스 문장: avez vous deja dine ?


In [61]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target


In [62]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
[['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
[['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


In [63]:
#정수 인코딩
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")


In [64]:
print("인코더 입력의 크기:", encoder_input.shape)
print("디코더 입력의 크기:", decoder_input.shape)
print("디코더 레이블의 크기:", decoder_target.shape)

인코더 입력의 크기: (33000, 7)
디코더 입력의 크기: (33000, 16)
디코더 레이블의 크기: (33000, 16)


In [65]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1

print("영어 단어 집합의 크기:", src_vocab_size)
print("프랑스어 단어 집합의 크기:", tar_vocab_size)

영어 단어 집합의 크기: 4497
프랑스어 단어 집합의 크기: 7894


In [66]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word

tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [67]:
#데이터 섞기
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[24767 22548 14340 ... 30232 20781 22067]


In [68]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [69]:
encoder_input[30997]

array([   5,   81,   13,    9, 4411,    1,    0], dtype=int32)

In [70]:
decoder_input[30997]

array([   2,    7,   27,    5,   11, 7641,    1,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [71]:
decoder_target[30997]

array([   7,   27,    5,   11, 7641,    1,    3,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [72]:
#훈련데이터의 10%를 테스트 데이터로 분리
n_of_val = int(33000*0.1)
print("검증 데이터의 개수:", n_of_val)

검증 데이터의 개수: 3300


In [73]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:,]
decoder_input_test = decoder_input[-n_of_val:,]
decoder_target_test = decoder_target[-n_of_val:]

In [74]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)


훈련 source 데이터의 크기 : (29700, 7)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 7)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


## 기계 번역기 만들기

In [76]:
from tensorflow.keras.layers import Dense, Embedding, Input, LSTM, Masking
from tensorflow.keras.models import Model

embedding_dim = 64
hidden_units = 64

In [78]:
#인코더

encoder_inputs = Input(shape = (None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)
enc_masking = Masking(mask_value = 0.0)(enc_emb) #패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [81]:
#디코더
decoder_inputs = Input(shape= (None,))
dec_emb_layer = Embedding(tar_vocab_size, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value = 0.0)(dec_emb)

decoder_lstm = LSTM(hidden_units, return_sequences = True, return_state = True)

decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

decoder_dense = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

In [83]:
model.fit(
    x = [encoder_input_train, decoder_input_train],
    y = decoder_target_train,
    validation_data = ([encoder_input_test, decoder_input_test],decoder_target_test),
    batch_size = 128,
    epochs = 20
)

Epoch 1/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - acc: 0.6050 - loss: 5.0592 - val_acc: 0.6205 - val_loss: 2.0376
Epoch 2/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - acc: 0.6476 - loss: 1.9374 - val_acc: 0.7387 - val_loss: 1.7026
Epoch 3/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - acc: 0.7480 - loss: 1.6413 - val_acc: 0.7508 - val_loss: 1.5472
Epoch 4/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - acc: 0.7567 - loss: 1.5013 - val_acc: 0.7705 - val_loss: 1.4337
Epoch 5/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - acc: 0.7764 - loss: 1.3831 - val_acc: 0.7824 - val_loss: 1.3420
Epoch 6/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - acc: 0.7903 - loss: 1.2906 - val_acc: 0.7998 - val_loss: 1.2574
Epoch 7/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - acc: 0.8059 - loss: 1.2040 - val_acc: 0.8099 - val_loss: 1.1982
Epoch 8/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - acc: 0.8156 - loss: 1.1375 - val_acc: 0.8178 - val_loss: 1.1520
Epoch 9/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 10s

## seq2seq 기계번역기 동작시키기

In [85]:
#인코더
encoder_model = Model(encoder_inputs, encoder_states)

#디코더
decoder_state_input_h = Input(shape = (hidden_units,))
decoder_state_input_c = Input(shape = (hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)

In [89]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence


In [90]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence


In [91]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
입력문장 : i m positive . 
정답문장 : j en suis sur . 
번역문장 : j en suis en train de travailler . 
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
입력문장 : these are fresh . 
정답문장 : celles ci sont fraiches . 
번역문장 : ils sont bien . 
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/

In [92]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
입력문장 : then what ? 
정답문장 : et alors ? 
번역문장 : qu est ce que ca va ? 
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
입력문장 : you re important . 
정답문장 : vous etes importante . 
번역문장 : vous etes es grande . 
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━

# BELU SCORE


In [93]:
import numpy as np
from collections import Counter
from nltk import ngrams

In [94]:
def simple_count(tokens, n):
    return Counter(ngrams(tokens, n))

In [95]:
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = candidate.split()
result = simple_count(tokens, 1)
print("유니그램 카운트", result)

유니그램 카운트 Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [96]:
candidate = 'the the the the the the the'
tokens = candidate.split()
result = simple_count(tokens, 1)
print(result)

Counter({('the',): 7})


In [100]:
def count_clip(candidate, reference_list, n):
  # Ca 문장에서 n-gram 카운트
  ca_cnt = simple_count(candidate, n)
  max_ref_cnt_dict = dict()

  for ref in reference_list:
    # Ref 문장에서 n-gram 카운트
    ref_cnt = simple_count(ref, n)

    # 각 Ref 문장에 대해서 비교하여 n-gram의 최대 등장 횟수를 계산.
    for n_gram in ref_cnt:
      if n_gram in max_ref_cnt_dict:
        max_ref_cnt_dict[n_gram] = max(ref_cnt[n_gram], max_ref_cnt_dict[n_gram])
      else:
        max_ref_cnt_dict[n_gram] = ref_cnt[n_gram]

  return {
        # count_clip = min(count, max_ref_count)
        n_gram: min(ca_cnt.get(n_gram, 0), max_ref_cnt_dict.get(n_gram, 0)) for n_gram in ca_cnt
     }


In [101]:
candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]
result = count_clip(candidate.split(), list(map(lambda ref: ref.split(), references)), 1)

print(result)

{('the',): 2}


In [102]:
#보정된 정밀도
def modified_precision(candidate, reference_list, n):
    clip_cnt = count_clip(candidate, reference_list, n)
    total_clip_cnt = sum(clip_cnt.values())

    cnt = simple_count(candidate, n)
    total_cnt = sum(cnt.values())

    if total_cnt == 0:
        total_cnt = 1

    return (total_clip_cnt/ total_cnt)

In [103]:
result = modified_precision(candidate.split(), list(map(lambda ref: ref.split(), references)), n=1)
print(result)

0.2857142857142857


In [121]:
# Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
def closest_ref_length(candidate, reference_list):
  ca_len = len(candidate) # ca 길이
  ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
  # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
  closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
  return closest_ref_len


In [122]:
#BP
def brevity_penalty(candidate, reference_list):
  ca_len = len(candidate)
  ref_len = closest_ref_length(candidate, reference_list)

  if ca_len > ref_len:
    return 1

  # candidate가 비어있다면 BP = 0 → BLEU = 0.0
  elif ca_len == 0 :
    return 0
  else:
    return np.exp(1 - ref_len/ca_len)


In [123]:
def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
  bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

  p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights,start=1)]
  # p1, p2, p3, ..., pn
  score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
  return bp * np.exp(score)


In [124]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('실습 코드의 BLEU :',bleu_score(candidate.split(),list(map(lambda ref: ref.split(), references))))
print('패키지 NLTK의 BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))


실습 코드의 BLEU : 0.5045666840058485
패키지 NLTK의 BLEU : 0.5045666840058485
